In [10]:
import pandas as pd
import os
df = pd.read_csv("/home/zo2151/imagenette/noisy_imagewoof.csv")
df_train = df.iloc[:9469,[0,2]]
df_test = df.iloc[9469:,:2]
df_train.to_csv("/home/zo2151/imagewoof2-160/train/train.csv")
df_test.to_csv("/home/zo2151/imagewoof2-160/val/val.csv")
ptrain = []
ltrain = []
for i in range(9469):
    filename = '/home/zo2151/imagewoof2-160/'+df.iloc[i,0]
    filename = filename[:-4]
    filename = filename + "jpeg"
    ptrain.append(filename)
    ltrain.append(df.iloc[i,1])
pval = []
lval = []
for i in range(9469,df.shape[0]):
    filename = '/home/zo2151/imagewoof2-160/'+df.iloc[i,0]
    filename = filename[:-4]
    filename = filename + "jpeg"
    pval.append(filename)
    lval.append(df.iloc[i,1])

In [11]:
import torch
import numpy as np
from datasets import load_dataset, load_metric
from transformers import ViTFeatureExtractor, ViTForImageClassification, TrainingArguments, Trainer
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import (CenterCrop, 
                                    Compose, 
                                    Normalize, 
                                    RandomHorizontalFlip,
                                    RandomResizedCrop, 
                                    Resize, 
                                    ToTensor)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
_train_transforms = Compose(
        [
            RandomResizedCrop(128),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )
_val_transforms = Compose(
        [
            Resize(128),
            CenterCrop(128),
            ToTensor(),
            normalize,
        ]
    )
trainset = load_dataset(path = "/home/zo2151/imagewoof2-160/train", data_files = ptrain)
valset = load_dataset(path = "/home/zo2151/imagewoof2-160/val", data_files = pval)
def train_transforms(examples):
    examples['pixel_values'] = [_train_transforms(image.convert("RGB")) for image in examples['image']]
    return examples

def val_transforms(examples):
    examples['pixel_values'] = [_val_transforms(image.convert("RGB")) for image in examples['image']]
    return examples

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}


loading feature extractor configuration file https://huggingface.co/google/vit-base-patch16-224-in21k/resolve/main/preprocessor_config.json from cache at /home/zo2151/.cache/huggingface/transformers/7c7f3e780b30eeeacd3962294e5154788caa6d9aa555ed6d5c2f0d2c485eba18.c322cbf30b69973d5aae6c0866f5cba198b5fe51a2fe259d2a506827ec6274bc
Feature extractor ViTFeatureExtractor {
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "ViTFeatureExtractor",
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "size": 224
}



Resolving data files:   0%|          | 0/9469 [00:00<?, ?it/s]

Using custom data configuration train-e6f06bbfc2b8d6c5
Reusing dataset image_folder (/home/zo2151/.cache/huggingface/datasets/image_folder/train-e6f06bbfc2b8d6c5/0.0.0/ee92df8e96c6907f3c851a987be3fd03d4b93b247e727b69a8e23ac94392a091)


  0%|          | 0/1 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/3485 [00:00<?, ?it/s]

Using custom data configuration val-c2a0cca0a61745c6
Reusing dataset image_folder (/home/zo2151/.cache/huggingface/datasets/image_folder/val-c2a0cca0a61745c6/0.0.0/ee92df8e96c6907f3c851a987be3fd03d4b93b247e727b69a8e23ac94392a091)


  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
trainset1 = trainset["train"]
test_ds = valset["train"]
splits = trainset1.train_test_split(test_size=0.1)
train_ds = splits['train']
val_ds = splits['test']

Loading cached split indices for dataset at /home/zo2151/.cache/huggingface/datasets/image_folder/train-e6f06bbfc2b8d6c5/0.0.0/ee92df8e96c6907f3c851a987be3fd03d4b93b247e727b69a8e23ac94392a091/cache-7e46934a8d570d4b.arrow and /home/zo2151/.cache/huggingface/datasets/image_folder/train-e6f06bbfc2b8d6c5/0.0.0/ee92df8e96c6907f3c851a987be3fd03d4b93b247e727b69a8e23ac94392a091/cache-8c8491eb0ed97bea.arrow


In [14]:
train_ds.set_transform(train_transforms)
val_ds.set_transform(val_transforms)
test_ds.set_transform(val_transforms)

In [15]:
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k',
                                                  num_labels=10,
                                                )

loading configuration file https://huggingface.co/google/vit-base-patch16-224-in21k/resolve/main/config.json from cache at /home/zo2151/.cache/huggingface/transformers/7bba26dd36a6ff9f6a9b19436dec361727bea03ec70fbfa82b70628109163eaa.92995a56e2eabab0c686015c4ad8275b4f9cbd858ed228f6a08936f2c31667e7
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224-in21k",
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,


In [16]:
metric_name = "accuracy"
args = TrainingArguments(
    f"test-cifar-10",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_dir='logs',
    remove_unused_columns=False,
)

metric = load_metric(metric_name)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=feature_extractor,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 8522
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 402


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.157283,0.902851
2,No log,0.794867,0.913411
3,No log,0.716033,0.928194


***** Running Evaluation *****
  Num examples = 947
  Batch size = 64
Saving model checkpoint to test-cifar-10/checkpoint-134
Configuration saved in test-cifar-10/checkpoint-134/config.json
Model weights saved in test-cifar-10/checkpoint-134/pytorch_model.bin
Feature extractor saved in test-cifar-10/checkpoint-134/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 947
  Batch size = 64
Saving model checkpoint to test-cifar-10/checkpoint-268
Configuration saved in test-cifar-10/checkpoint-268/config.json
Model weights saved in test-cifar-10/checkpoint-268/pytorch_model.bin
Feature extractor saved in test-cifar-10/checkpoint-268/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 947
  Batch size = 64
Saving model checkpoint to test-cifar-10/checkpoint-402
Configuration saved in test-cifar-10/checkpoint-402/config.json
Model weights saved in test-cifar-10/checkpoint-402/pytorch_model.bin
Feature extractor saved in test-cifar-10/checkpoint-402/pr

TrainOutput(global_step=402, training_loss=1.1972861977952036, metrics={'train_runtime': 151.4398, 'train_samples_per_second': 168.82, 'train_steps_per_second': 2.655, 'total_flos': 1.9813022585849364e+18, 'train_loss': 1.1972861977952036, 'epoch': 3.0})

In [17]:
outputs = trainer.predict(test_ds)
print(outputs.metrics)

***** Running Prediction *****
  Num examples = 3485
  Batch size = 64


{'test_loss': 0.7450010776519775, 'test_accuracy': 0.9202295552367289, 'test_runtime': 11.7137, 'test_samples_per_second': 297.516, 'test_steps_per_second': 4.695}
